In [1]:
from dotenv import load_dotenv
from langchain.chains.question_answering.map_rerank_prompt import output_parser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_pinecone import PineconeVectorStore

load_dotenv()

True

## 파일로드

In [8]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import PyPDFLoader

dur_pregnant = CSVLoader('./data/DUR_pregnant.csv')
commandment_10 = PyPDFLoader('./data/commandment_10.pdf')
docs = PyPDFLoader('./data/docs.pdf')
drug_for = PyPDFLoader('./data/drug_for.pdf')
korean_mothersafe = PyPDFLoader('./data/korean_mothersafe.pdf')

dur_pregnant_docs = dur_pregnant.load()
commandment_10_docs = commandment_10.load()
docs_docs = docs.load()
drug_for.docs = docs_docs
korean_mothersafe.docs = korean_mothersafe.load()

In [29]:
documents = commandment_10_docs + docs_docs + drug_for.docs + korean_mothersafe.docs

33

In [19]:
# 개행 문자 제거
for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')
documents

[Document(metadata={'source': './data/commandment_10.pdf', 'page': 0}, page_content='1    ■ 임산부의 안전하고, 효과적인 약 복용을 위한 약물 요법 10계명  ■ 10 Mga Gabay para sa Ligtas at Epektibong Medikasyon  ng mga Babaeng  Nagbubuntis    1. 임신 가능한 여성의 경우 약물 복용 전 임신 여부 확인하십시오.  임신 가능한 여성이 약물을 복용할 때는 임신 계획 및 임신 여부를 반드시  확인하여야 하며 전문가와 상의 후 투여하는 것이 필요합니다.   1. Alamin kung nagdadalantao o hindi bago uminom ng anumang gamot.  Kailangang siguruhin ng isang babaeng ma y kakayanang magkaanak kung nagbubuntis o  hindi at kung makakapagsilang ng sanggol, at kumunsulta sa isang espesyalista sa panahon  ng medikasyon.    2. 임신 중 약물 복용은 태아에게 영향을 줄 수 있습니다.  임산부가 약물 복용 시 주의해야 하는 까닭은 약물이 태반을 통과하여 태아에게  나쁜 영향을 미칠 수 있고 임산부에게도 독성을 일으킬 수 있기 때문입니다  2. Ang medikasyon sa panahon ng pagbubuntis ay maaring makaapekto sa sanggol.  Ang mga babaeng nagdadalantaong sumasailalim sa anumang medikasyon ay kailangan ng  matinding p ag-iingat sa dahilang ang mga gamot ay maaring makaapekto sa sanggol sa  pamamagitan ng  ‘placental barrier’. 

## Text Spliter

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

spliter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs = spliter.split_documents(documents)
print(len(docs))
docs

52


[Document(metadata={'source': './data/commandment_10.pdf', 'page': 0}, page_content='1    ■ 임산부의 안전하고, 효과적인 약 복용을 위한 약물 요법 10계명  ■ 10 Mga Gabay para sa Ligtas at Epektibong Medikasyon  ng mga Babaeng  Nagbubuntis    1. 임신 가능한 여성의 경우 약물 복용 전 임신 여부 확인하십시오.  임신 가능한 여성이 약물을 복용할 때는 임신 계획 및 임신 여부를 반드시  확인하여야 하며 전문가와 상의 후 투여하는 것이 필요합니다.   1. Alamin kung nagdadalantao o hindi bago uminom ng anumang gamot.  Kailangang siguruhin ng isang babaeng ma y kakayanang magkaanak kung nagbubuntis o  hindi at kung makakapagsilang ng sanggol, at kumunsulta sa isang espesyalista sa panahon  ng medikasyon.    2. 임신 중 약물 복용은 태아에게 영향을 줄 수 있습니다.  임산부가 약물 복용 시 주의해야 하는 까닭은 약물이 태반을 통과하여 태아에게  나쁜 영향을 미칠 수 있고 임산부에게도 독성을 일으킬 수 있기 때문입니다  2. Ang medikasyon sa panahon ng pagbubuntis ay maaring makaapekto sa sanggol.  Ang mga babaeng nagdadalantaong sumasailalim sa anumang medikasyon ay kailangan ng  matinding p ag-iingat sa dahilang ang mga gamot ay maaring makaapekto sa sanggol sa  pamamagitan ng  ‘placental barrier’. 

## 임베딩 모델

In [23]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model = 'text-embedding-3-small'
)

In [30]:
from langchain_chroma.vectorstores import Chroma

vector_store = Chroma.from_documents(docs, embeddings)

In [35]:
retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k':3}
)

query = '임산부에게 해로운 성분은?'

retrievals = retriever.batch([query])
retrievals

[[Document(metadata={'page': 12, 'source': './data/korean_mothersafe.pdf'}, page_content='임산부의 약물 상담 지침  11 그림 8. 마더세이프전문상담콜센터: 상담종류별 분류 (2010.4-2010.9) 그림 9. 마더세이프전문상담콜센터: 노출종류별 분류 (2010.4-2010.9) 임신부 83% 계획임신부 8% 모유수유부 9% 90% 3% 2% 1% 1% 1% 0% 0% 2% 약물 유해물질 방사선 음주 고열 감염 흡연 유전관련 기타 N=1,798'),
  Document(metadata={'page': 12, 'source': './data/korean_mothersafe.pdf'}, page_content='임산부의 약물 상담 지침  11 그림 8. 마더세이프전문상담콜센터: 상담종류별 분류 (2010.4-2010.9) 그림 9. 마더세이프전문상담콜센터: 노출종류별 분류 (2010.4-2010.9) 임신부 83% 계획임신부 8% 모유수유부 9% 90% 3% 2% 1% 1% 1% 0% 0% 2% 약물 유해물질 방사선 음주 고열 감염 흡연 유전관련 기타 N=1,798'),
  Document(metadata={'page': 4, 'source': './data/korean_mothersafe.pdf'}, page_content='임산부의 약물 상담 지침 그림 1. 임신중 약물의 위험성. 약물들의 대부분은 대기형발생위험률이 기본위험률(1-3%)을 넘지 않는다. 단지 몇 가지 약물만이 20~30%의 위험률을 가짐. FDA 분류를 임신인지 모르고 약물에 노출된 임신부에게 약물의 안전성 및 위험성 평가를 위해 적용하는 것은 옳은가? 옳지 않다. 미국 FDA 분류는 1979년에 임신부의 치료지침을 제공하기 위해 개발되었지만, 이상적이지 않은 것으로 평가되고 있다. 대부분의 약물의 분류가 동물실험자료, 증례발표, 그리고 사람에 대한 자료가 제한되어 있거나 거의 없는 상태에 근거하고 있다. 그리고 최근 약

# Prompt 생성

In [37]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system', '''
너는 산모(임산부)와 태아의 건강과 안전을 최우선으로 고려하는 의료 전문가다.
예시1:
산모에게 의약품을 설명할 때 다음 사항을 반드시 포함하라
1. 약물의 이름과 주요 성분
2. 약물이 사용되는 주요 목적과 효과
3. 임산부에게 약물이 권장되거나 제한되는 이유
4. 가능한 부작용 및 태아에 미칠 수 있는 영향
5. 약물을 사용하는 동안 주의해야 할 사항
6. 약물 사용을 대체할 수 있는 안전한 대안

정보를 제공할 때는 산모가 이해하기 쉽게 쉬운 용어를 사용하고, 불필요한 걱정을 피하면서도 반드시 중요한 위험 요소는 명확히 전달하라.
예시 응답: [약물 이름], 이 약물은 [효능 설명]을 위해 사용되며, 임산부에게는 [권장/주의/금지]됩니다. 이유는 [상세 설명]입니다. 만약 사용해야 한다면, 반드시 [의료진과 상의/정확한 복용 지침]을 따라야 합니다. 부작용으로는 [부작용 예시]가 있을 수 있습니다.

예시2:
너는 임산부를 대상으로 하는 약물 상담 전문가로, 특정 의약품에 대해 다음 질문에 답변해야 한다
1. 약물은 무엇이고 어떤 용도로 사용되는가?
2. 임신 중 이 약물의 사용이 권장되는가? (안전성 등급 포함)
3. 임신 중 이 약물 복용 시 주의해야 할 사항은 무엇인가?
4. 약물 복용 시 태아에게 미칠 수 있는 위험은 무엇인가?
5. 이 약물 대신 임산부가 사용할 수 있는 대체 방법은 무엇인가?

예시 출력:
파라세타몰(아세트아미노펜)은 일반적으로 통증 완화와 발열을 낮추기 위해 사용되는 약물입니다.
임신 중에는 비교적 안전하다고 간주되며, 필요한 경우 짧은 기간 동안 사용이 가능합니다.
그러나 권장 복용량을 초과하지 않도록 주의해야 하며, 사용 전에 반드시 의사와 상의하세요.
부작용으로는 간에 부담을 줄 수 있으며, 드물게 태아에 영향을 미칠 가능성도 보고된 바 있습니다.
통증 관리의 대안으로는 충분한 휴식, 온찜질, 또는 가벼운 운동이 추천될 수 있습니다.
더 자세한 정보는 담당 의사와 상의하시기 바랍니다
    '''),
    ('user', '''사용자의 질문에 context만을 이용해 답변해주세요.
    
    question = {query}
    
    context = {context}
    ''')
    
])

In [38]:
prompt.invoke({'query' : query, 'context':retriever})


ChatPromptValue(messages=[SystemMessage(content='\n너는 산모(임산부)와 태아의 건강과 안전을 최우선으로 고려하는 의료 전문가다.\n예시1:\n산모에게 의약품을 설명할 때 다음 사항을 반드시 포함하라\n1. 약물의 이름과 주요 성분\n2. 약물이 사용되는 주요 목적과 효과\n3. 임산부에게 약물이 권장되거나 제한되는 이유\n4. 가능한 부작용 및 태아에 미칠 수 있는 영향\n5. 약물을 사용하는 동안 주의해야 할 사항\n6. 약물 사용을 대체할 수 있는 안전한 대안\n\n정보를 제공할 때는 산모가 이해하기 쉽게 쉬운 용어를 사용하고, 불필요한 걱정을 피하면서도 반드시 중요한 위험 요소는 명확히 전달하라.\n예시 응답: [약물 이름], 이 약물은 [효능 설명]을 위해 사용되며, 임산부에게는 [권장/주의/금지]됩니다. 이유는 [상세 설명]입니다. 만약 사용해야 한다면, 반드시 [의료진과 상의/정확한 복용 지침]을 따라야 합니다. 부작용으로는 [부작용 예시]가 있을 수 있습니다.\n\n예시2:\n너는 임산부를 대상으로 하는 약물 상담 전문가로, 특정 의약품에 대해 다음 질문에 답변해야 한다\n1. 약물은 무엇이고 어떤 용도로 사용되는가?\n2. 임신 중 이 약물의 사용이 권장되는가? (안전성 등급 포함)\n3. 임신 중 이 약물 복용 시 주의해야 할 사항은 무엇인가?\n4. 약물 복용 시 태아에게 미칠 수 있는 위험은 무엇인가?\n5. 이 약물 대신 임산부가 사용할 수 있는 대체 방법은 무엇인가?\n\n예시 출력:\n파라세타몰(아세트아미노펜)은 일반적으로 통증 완화와 발열을 낮추기 위해 사용되는 약물입니다.\n임신 중에는 비교적 안전하다고 간주되며, 필요한 경우 짧은 기간 동안 사용이 가능합니다.\n그러나 권장 복용량을 초과하지 않도록 주의해야 하며, 사용 전에 반드시 의사와 상의하세요.\n부작용으로는 간에 부담을 줄 수 있으며, 드물게 태아에 영향을 미칠 가능성도 보고된 바 있습니다.\n통증 관리의 대안으로는 충분한 휴식, 

## 모델 생성

In [39]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = 'gpt-4o',
    temperature=0.5
)

## chain 생성

In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = {'query' : RunnablePassthrough(), 'context':retriever} | prompt | model | StrOutputParser()

chain.invoke('임신 계획 중에 복용하면 안되는 약에 대해 설명해줘')

'임신 계획 중에 복용하면 안 되는 약물로는 메토트렉세이트가 있으며, 이는 주요 기형 유발 물질로 임신 초기에 사용 시 무뇌아, 두개골형성장애 등을 유발할 수 있습니다. 따라서 임신을 계획하고 있는 경우 이 약물의 사용을 피해야 합니다. 만약 이미 복용 중이라면, 약물이 체내에서 제거되는 시간을 고려하여 약 3개월 후에 임신을 시도하는 것이 좋습니다. 이 외에도 소염진통제, 코르티코스테로이드, 면역억제제는 상대적으로 낮은 위험을 가지고 있으나, 임신 시 사용 여부는 질병 상태에 따라 달라질 수 있습니다. 임신을 계획 중이라면 반드시 전문가와 상의하여 안전한 약물 사용 계획을 세우는 것이 중요합니다.'